# The Burguer's Equation

$$
\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}
$$

In [4]:
import math 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn

In [5]:
class NN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super(NN,self).__init__(*args, **kwargs)
        self.net = torch.nn.Sequential(
            nn.Linear(2,20),
            nn.Tanh(),
            nn.Linear(20,30),
            nn.Tanh(),
            nn.Linear(30,30),
            nn.Tanh(),
            nn.Linear(30,20),
            nn.Tanh(),
            nn.Linear(20,20),
            nn.Tanh(),
            nn.Linear(20,1),
        )
    
    def forward(self,x):
        out = self.net(x)
        return out

In [9]:
x = torch.arange(-1,1+0.1,0.1)
t = torch.arange(0,1+0.1,0.1)

print(x)
print(t)


tensor([-1.0000e+00, -9.0000e-01, -8.0000e-01, -7.0000e-01, -6.0000e-01,
        -5.0000e-01, -4.0000e-01, -3.0000e-01, -2.0000e-01, -1.0000e-01,
        -2.9802e-09,  1.0000e-01,  2.0000e-01,  3.0000e-01,  4.0000e-01,
         5.0000e-01,  6.0000e-01,  7.0000e-01,  8.0000e-01,  9.0000e-01,
         1.0000e+00])
tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
        0.9000, 1.0000])


In [11]:
X = torch.stack(torch.meshgrid(x,t)).reshape(2,-1).T
print(X)

tensor([[-1.0000e+00,  0.0000e+00],
        [-1.0000e+00,  1.0000e-01],
        [-1.0000e+00,  2.0000e-01],
        [-1.0000e+00,  3.0000e-01],
        [-1.0000e+00,  4.0000e-01],
        [-1.0000e+00,  5.0000e-01],
        [-1.0000e+00,  6.0000e-01],
        [-1.0000e+00,  7.0000e-01],
        [-1.0000e+00,  8.0000e-01],
        [-1.0000e+00,  9.0000e-01],
        [-1.0000e+00,  1.0000e+00],
        [-9.0000e-01,  0.0000e+00],
        [-9.0000e-01,  1.0000e-01],
        [-9.0000e-01,  2.0000e-01],
        [-9.0000e-01,  3.0000e-01],
        [-9.0000e-01,  4.0000e-01],
        [-9.0000e-01,  5.0000e-01],
        [-9.0000e-01,  6.0000e-01],
        [-9.0000e-01,  7.0000e-01],
        [-9.0000e-01,  8.0000e-01],
        [-9.0000e-01,  9.0000e-01],
        [-9.0000e-01,  1.0000e+00],
        [-8.0000e-01,  0.0000e+00],
        [-8.0000e-01,  1.0000e-01],
        [-8.0000e-01,  2.0000e-01],
        [-8.0000e-01,  3.0000e-01],
        [-8.0000e-01,  4.0000e-01],
        [-8.0000e-01,  5.000

/home/thiago/anaconda3/envs/torch_gpu/lib/python3.9/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025831482/work/aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
class Net:
    def __init__(self) -> None:
        device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )

        self.h = 0.1
        self.k = 0.1 

        x = torch.arange(-1,1+self.h,self.h)
        t = torch.arange(0,1+self.k,self.k)

        self.X = torch.stack(torch.meshgrid(x,t)).reshape(2,-1).T

        # Boundary condition
        bc1 = torch.stack(torch.meshgrid(x[0],t)).reshape(2,-1).T
        bc2 = torch.stack(torch.meshgrid(x[-1],t)).reshape(2,-1).T

        # Initial condition
        ic = torch.stack(torch.meshgrid(x,t[0])).reshape(2,-1).T
        
        self.X_train = torch.cat([bc1,bc2,ic])

        tg_bc1 = torch.zeros(len(bc1))
        tg_bc2 = torch.zeros(len(bc2))
        tg_ic = -torch.sin(math.pi*ic[:,0])

        self.tg_train = torch.cat([tg_bc1,tg_bc2,tg_ic])
        self.tg_train = self.tg_train.unsqueeze(1) 
